In [4]:
import requests
import re
from bs4 import BeautifulSoup

def carjam_colour(plate: str) -> str:
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'
    }
    
    r = requests.get('https://www.carjam.co.nz/car/?plate=' + plate.upper(), headers=headers).text
    try:
        soup = BeautifulSoup(r, 'lxml')
        car_colour_html = str(soup.findAll('span', {'class': 'value'})[3])
        car_colour = re.findall( r'>(.*?)<' , car_colour_html)[0]
        if not car_colour: 
            raise ValueError
        else:
            return car_colour.lower()
    except ValueError:
        print('Plate: {}, no valid entry found'.format(plate))
        return None

In [ ]:
import os
import database
import carjam_soup
from model_eval import predict_image

def get_filenames(path: str) -> list:
    filenames_list = []
    for filename in os.listdir(path):
        file = os.path.join(path,filename)
        if os.path.isfile(file):
            filenames_list.append(file)
    return filenames_list


def get_predictions(filename_list: list) -> list:
    predictions = []
    for f in filename_list:
        predictions.append(predict_image(f))
    return predictions

def compare_carjam(predictions: list) -> list:
    db = database.retreive_all()
    for i in range(len(predictions) - 1):
        predictions[i]['plate_read'] = db[i]
    
    for entry in predictions[:len(predictions) - 1]:
        entry_plate = entry['plate_read'][1]
        carj_colour = carjam_soup.carjam_colour(entry_plate)
        if carj_colour is not None:
            if carj_colour == entry['colour']:
                entry['Match'] = True
            else:
                entry['Match'] = False
    
    return predictions

def display(predictions: list):
    predictions = predictions[:-1]
    for pr in predictions:
        print("Plate Read: {}, Predicted Colour: {}, Match: {}".format(
            pr['plate_read'][1],
            pr['colour'], 
            pr['Match'])
            )

In [6]:
def main():
    filenames = get_filenames('images')
    predictions = compare_carjam(get_predictions(filenames))
    display(predictions)

   
if __name__ == '__main__':
    main()

Plate Read: gwt215, Predicted Colour: white, Match: True
Plate Read: gwt215, Predicted Colour: white, Match: True
Plate Read: gwt215, Predicted Colour: beige, Match: False
Plate Read: hrz326, Predicted Colour: blue, Match: False
Plate Read: hrz326, Predicted Colour: brown, Match: False
Plate Read: jsg421, Predicted Colour: brown, Match: False
Plate Read: jsg421, Predicted Colour: blue, Match: False
Plate Read: jsg421, Predicted Colour: grey, Match: True
Plate Read: w0000f, Predicted Colour: grey, Match: False
Plate Read: w0000f, Predicted Colour: blue, Match: False
Plate Read: lbk263, Predicted Colour: white, Match: True
Plate Read: lbk263, Predicted Colour: beige, Match: False
Plate Read: 69syc, Predicted Colour: white, Match: False
